In [1]:
# Importing all necessary libraries
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
 
img_width, img_height = 224, 224

2022-01-28 17:56:00.462958: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-28 17:56:00.462985: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [7]:
train_data_dir = 'data/v_data/train'
validation_data_dir = 'data/v_data/test'
nb_train_samples =400
nb_validation_samples = 100
epochs = 10
batch_size = 16

In [3]:

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [58]:
model = Sequential()
model.add(Conv2D(32, (2, 2), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Conv2D(32, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [59]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [60]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
 
test_datagen = ImageDataGenerator(rescale=1. / 255)
 
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')
 
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')
 
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Found 400 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


/tmp/ipykernel_181822/3119093969.py:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/10
25/25 [==============================] - 19s 714ms/step - loss: 0.8328 - accuracy: 0.6375 - val_loss: 0.5343 - val_accuracy: 0.6562
Epoch 2/10
25/25 [==============================] - 19s 769ms/step - loss: 0.5548 - accuracy: 0.7500 - val_loss: 0.3812 - val_accuracy: 0.9062
Epoch 3/10
25/25 [==============================] - 19s 731ms/step - loss: 0.4213 - accuracy: 0.8225 - val_loss: 0.4731 - val_accuracy: 0.7396
Epoch 4/10
25/25 [==============================] - 18s 715ms/step - loss: 0.4314 - accuracy: 0.8175 - val_loss: 0.3717 - val_accuracy: 0.8438
Epoch 5/10
25/25 [==============================] - 18s 705ms/step - loss: 0.3548 - accuracy: 0.8550 - val_loss: 0.3067 - val_accuracy: 0.8854
Epoch 6/10
25/25 [==============================] - 18s 727ms/step - loss: 0.3083 - accuracy: 0.8750 - val_loss: 0.2803 - val_accuracy: 0.8750
Epoch 7/10
25/25 [==============================] - 19s 749ms/step - loss: 0.3459 - accuracy: 0.8725 - val_loss: 0.4280 - val_accuracy: 0.8021

In [61]:
model.save('saved_model.h5')

In [64]:
from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
import numpy as np

image = load_img('data/v_data/test/planes/5.jpg', target_size=(224, 224))
img = np.array(image)
img = img / 255.0
img = img.reshape(1,224,224,3)
label = model.predict(img)
print("Predicted Class (0 - Cars , 1- Planes): ", label[0][0])

Predicted Class (0 - Cars , 1- Planes):  0.6901742


In [65]:
saved_model = load_model('saved_model.h5')
 
image = load_img('data/v_data/test/planes/5.jpg', target_size=(224, 224))
img = np.array(image)
img = img / 255.0
img = img.reshape(1,224,224,3)
label = saved_model.predict(img)
print("Predicted Class (0 - Cars , 1- Planes): ", label[0][0])

Predicted Class (0 - Cars , 1- Planes):  0.6901742


In [136]:
import os
import pandas as pd

submission_df = pd.DataFrame()
submission_df["ID"] = []
submission_df["predict"] = []

car_dir = "data/v_data/test/cars/"
for i, path in enumerate(os.listdir(car_dir)):
    image = load_img(os.path.join(car_dir, path), target_size=(224, 224))
    img = np.array(image)
    img = img / 255.0
    img = img.reshape(1,224,224,3)
    label = saved_model.predict(img)
    submission_df = submission_df.append(pd.Series([int(i+1),label[0][0]], index=["ID", "predict"]), ignore_index=True) # Creating a submission file
# submission_df["predict"].round(4) 
submission_df["ID"] = submission_df["ID"].astype(int)
submission_df.to_csv('test_car.csv', index = False)
submission_df.head()

,ID,predict
0,1,3.349811e-03
1,2,9.336472e-04
2,3,3.797412e-04
3,4,2.337992e-04
4,5,4.339977e-07


In [137]:
submission_df = pd.DataFrame()
submission_df["ID"] = []
submission_df["predict"] = []

plane_dir = "data/v_data/test/planes/"
for i, path in enumerate(os.listdir(plane_dir)):
    image = load_img(os.path.join(plane_dir, path), target_size=(224, 224))
    img = np.array(image)
    img = img / 255.0
    img = img.reshape(1,224,224,3)
    label = saved_model.predict(img)
    submission_df = submission_df.append(pd.Series([int(i+1),label[0][0]], index=["ID", "predict"]), ignore_index=True) # Creating a submission file
# submission_df["predict"].round(4) 
submission_df["ID"] = submission_df["ID"].astype(int)
submission_df.to_csv('test_plane.csv', index = False)
submission_df.head()

,ID,predict
0,1,0.114071
1,2,0.989791
2,3,0.964984
3,4,0.860722
4,5,0.960105


In [92]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 223, 223, 32)      416       
                                                                 
 activation_25 (Activation)  (None, 223, 223, 32)      0         
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 111, 111, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 110, 110, 32)      4128      
                                                                 
 activation_26 (Activation)  (None, 110, 110, 32)      0         
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 55, 55, 32)       0         
 g2D)                                                 

In [124]:
pd.reset_option("display.float_format")